# Divergenza KL e ELPD {#sec-kl-elpd}

**Prerequisiti**

**Concetti e competenze chiave**

- Comprendere in modo dettagliato il concetto di Expected Log Predictive Density (ELPD), apprezzandone l'importanza e l'applicabilità nel contesto della valutazione predittiva dei modelli statistici.
- Esplorare e analizzare la relazione tra il concetto di entropia e la ELPD, identificando come l'entropia possa influenzare o riflettere la capacità predittiva di un modello attraverso la densità logaritmica predittiva prevista.

**Preparazione del Notebook**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom
import scipy.stats as stats
from scipy.optimize import minimize
import arviz as az

In [3]:
seed: int = sum(map(ord, "kl"))
rng: np.random.Generator = np.random.default_rng(seed=seed)
az.style.use("arviz-darkgrid")
%config InlineBackend.figure_format = "retina"

## Introduzione {.unnumbered}

In questo capitolo, esamineremo in dettaglio tre concetti fondamentali per la valutazione e il confronto di modelli statistici nel contesto bayesiano: la distribuzione predittiva posteriori, 
la Divergenza di Kullback-Leibler ($\mathbb{KL}$) e la Densità Predittiva Logaritmica Attesa (Expected Log Predictive Density, ELPD). Questi strumenti ci permettono di quantificare l'adattamento dei modelli ai dati e la loro capacità predittiva.

## Distribuzione Predittiva Posteriori

La distribuzione predittiva posteriori $Q(\tilde{y} \mid y)$ rappresenta la distribuzione dei possibili nuovi dati $\tilde{y},$ dati i dati osservati $y$. Essa si ottiene combinando:

- la distribuzione del modello per i nuovi dati data una configurazione dei parametri $\theta$: $Q(\tilde{y} \mid \theta)$;
- la distribuzione posteriore dei parametri data i dati osservati: $P(\theta \mid y)$.

Matematicamente, la distribuzione predittiva posteriori può essere scritta come:

$$
Q(\tilde{y} \mid y) = \int Q(\tilde{y} \mid \theta) P(\theta \mid y) d\theta.
$$

Questa espressione si legge come l'integrale della distribuzione predittiva $Q(\tilde{y} \mid \theta)$ pesata dalla distribuzione posteriore dei parametri $P(\theta \mid y)$. In altre parole, stiamo "mediando" le previsioni del modello su tutte le possibili configurazioni dei parametri, tenendo conto della loro probabilità posteriore. La distribuzione predittiva posteriori è stata descritta nella @sec-posterior-predictive-distribution.

## Divergenza di Kullback-Leibler (KL)

La divergenza di Kullback-Leibler $\mathbb{KL}(P \parallel Q)$ misura la distanza tra due distribuzioni, $P$ e $Q$. È definita come:

$$
\mathbb{KL}(P \parallel Q) = \mathbb{E}_P[\log P(X)] - \mathbb{E}_P[\log Q(X)].
$$

Il termine $\mathbb{E}_P[\log P(X)]$ è il valore atteso del logaritmo della distribuzione $P(X)$ sotto $P$ stessa, cioè:

  $$
  \mathbb{E}_P[\log P(X)] = \sum_{i=1}^n p_i \log p_i.
  $$

  Per distribuzioni continue, l'espressione diventa:

  $$
  \mathbb{E}_P[\log P(X)] = \int p(x) \log p(x) \, dx.
  $$

  Questo termine rappresenta l'entropia negativa della distribuzione $P$.

Il termine $\mathbb{E}_P[\log Q(X)]$ è il valore atteso del logaritmo della distribuzione $Q(X)$ sotto la distribuzione $P(X)$:

  $$
  \mathbb{E}_P[\log Q(X)] = \sum_{i=1}^n p_i \log q_i.
  $$

  Per variabili continue:

  $$
  \mathbb{E}_P[\log Q(X)] = \int p(x) \log q(x) \, dx.
  $$

  Questo rappresenta l'entropia incrociata tra $P$ e $Q$.

La divergenza KL è quindi:

$$
\mathbb{KL}(P \parallel Q) = \sum_{i=1}^n p_i (\log p_i - \log q_i)
$$

o, in forma continua:

$$
\mathbb{KL}(P \parallel Q) = \int p(x) (\log p(x) - \log q(x)) \, dx
$$

Un punto importante è che la divergenza KL non è simmetrica, il che significa che $\mathbb{KL}(P \parallel Q) \neq \mathbb{KL}(Q \parallel P)$. Questa divergenza misura quanta "informazione" si perde usando $Q$ al posto di $P$.

::: {#exm-}

Per rendere più chiaro il concetto di divergenza di Kullback-Leibler (KL) nel caso discreto, consideriamo un esempio semplice. Immaginiamo di avere due distribuzioni di probabilità discrete, $P$ e $Q$, su un insieme di eventi $\{A, B, C\}$. Supponiamo che le probabilità associate a ciascun evento sotto le distribuzioni $P$ e $Q$ siano le seguenti:

- distribuzione $P$: $P(A) = 0.5$, $P(B) = 0.3$, $P(C) = 0.2$;
- distribuzione $Q$: $Q(A) = 0.4$, $Q(B) = 0.4$, $Q(C) = 0.2$.

La divergenza KL $\mathbb{KL}(P \parallel Q)$ è definita come:

$$
\mathbb{KL}(P \parallel Q) = \sum_{i} P(i) \log \frac{P(i)}{Q(i)},
$$

dove $i$ scorre su tutti gli eventi possibili $\{A, B, C\}$. Quindi, dobbiamo calcolare  $P(A) \log \frac{P(A)}{Q(A)}$,  $P(B) \log \frac{P(B)}{Q(B)}$ e  $P(C) \log \frac{P(C)}{Q(C)}$, ovvero,

   $$
   P(A) \log \frac{P(A)}{Q(A)} = 0.5 \log \frac{0.5}{0.4} 0.11155,
   $$

   $$
   P(B) \log \frac{P(B)}{Q(B)} = 0.3 \log \frac{0.3}{0.4} = -0.08631,
   $$

   $$
   P(C) \log \frac{P(C)}{Q(C)} = 0.2 \log \frac{0.2}{0.2} = 0.
   $$

Ora sommiamo tutti i termini:

$$
\mathbb{KL}(P \parallel Q) = 0.11155 + (-0.08631) + 0 = 0.02524.
$$

La divergenza KL $\mathbb{KL}(P \parallel Q) \approx 0.025$ indica che la distribuzione $Q$ non è troppo lontana dalla distribuzione $P$, ma c'è una piccola discrepanza tra le due. Se il valore fosse maggiore, significherebbe che la distribuzione $Q$ si discosta maggiormente da $P$, indicando una maggiore perdita di informazione se si usa $Q$ al posto di $P$.

:::

## Expected Log Predictive Density (ELPD)

L'ELPD è una misura che valuta la capacità predittiva di un modello. Si definisce come:

$$
\text{ELPD} = \sum_{i=1}^n \log p(y_i | \mathbf{y}_{-i})
$$

dove:

- $y_i$ è l'i-esima osservazione,
- $\mathbf{y}_{-i}$ rappresenta tutte le osservazioni eccetto $y_i$.

L'idea alla base dell'ELPD è di valutare quanto bene un modello prevede ciascun dato osservato utilizzando tutti gli altri dati. Questo approccio, noto come leave-one-out cross-validation, è particolarmente utile per evitare l'overfitting, perché valuta le previsioni del modello su dati che non ha "visto" durante il processo di addestramento.

::: {#exm-}

Per illustrare il calcolo dell'Expected Log Predictive Density (ELPD), vediamo un esempio semplice con un set di dati molto piccolo. Supponiamo di avere un dataset di tre osservazioni: $y_1, y_2, y_3$. Supponiamo che il nostro modello stimi le probabilità per ciascuna osservazione in base a tutte le altre osservazioni, cioè utilizziamo la leave-one-out cross-validation (LOO-CV) per calcolare $p(y_i \mid \mathbf{y}_{-i})$.

Immaginiamo che il modello produca le seguenti probabilità condizionali per ogni osservazione $y_i$:

- $p(y_1 \mid y_2, y_3) = 0.6$,
- $p(y_2 \mid y_1, y_3) = 0.7$,
- $p(y_3 \mid y_1, y_2) = 0.5$.

L'ELPD si calcola sommando i logaritmi di queste probabilità:

$$
\text{ELPD} = \log p(y_1 \mid y_2, y_3) + \log p(y_2 \mid y_1, y_3) + \log p(y_3 \mid y_1, y_2).
$$

Calcoliamo i logaritmi naturali di ciascuna probabilità:

- $\log p(y_1 \mid y_2, y_3) = \log 0.6 \approx -0.5108$,
- $\log p(y_2 \mid y_1, y_3) = \log 0.7 \approx -0.3567$,
- $\log p(y_3 \mid y_1, y_2) = \log 0.5 \approx -0.6931$.

Sommiamo i logaritmi per ottenere l'ELPD:

$$
\text{ELPD} = -0.5108 + (-0.3567) + (-0.6931) = -1.5606.
$$

L'ELPD ottenuto è $-1.5606$. In generale, valori più vicini a 0 o positivi indicano una migliore capacità predittiva del modello, poiché suggeriscono che le probabilità condizionali assegnate dal modello alle osservazioni lasciate fuori non sono troppo basse. Valori molto negativi indicherebbero che il modello ha assegnato probabilità molto basse alle osservazioni effettivamente osservate, suggerendo una scarsa capacità predittiva. L'ELPD è un modo efficace per valutare quanto bene un modello generalizza a nuovi dati, evitando l'overfitting.

:::

## Collegamento tra Divergenza KL e ELPD

Entrambe le misure (Divergenza KL e ELPD) valutano la qualità del modello, ma da prospettive diverse:

- la divergenza KL misura quanto la distribuzione predittiva del modello $Q$ si avvicina alla "vera" distribuzione $P$;
- l'ELPD valuta direttamente la capacità del modello di prevedere nuovi dati, utilizzando la tecnica leave-one-out.

Idealmente, vorremmo calcolare la divergenza KL per confrontare i modelli, poiché fornisce una misura diretta di quanto un modello si avvicina alla vera distribuzione dei dati. Tuttavia, poiché $P$ è sconosciuta nella pratica, la divergenza KL non può essere calcolata direttamente. Per questo motivo, nella pratica si utilizza l'ELPD, che, pur fornendo una misura leggermente diversa, ha il grande vantaggio di poter essere calcolata e offre una stima della capacità predittiva di un modello.

L'ELPD tende a favorire modelli che non solo si adattano bene ai dati osservati, ma che sono anche robusti rispetto all'overfitting, grazie alla sua enfasi sulla capacità predittiva su dati non osservati.

In sintesi, la distribuzione predittiva posteriori, la divergenza di Kullback-Leibler e l'ELPD sono strumenti matematici che ci permettono di valutare la bontà di un modello statistico. La divergenza KL fornisce una misura teoricamente ideale di quanto un modello approssima la vera distribuzione dei dati, mentre l'ELPD offre un'alternativa praticabile che valuta la capacità del modello di fare previsioni su nuovi dati. Questi concetti sono fondamentali nell'inferenza bayesiana, dove è importante non solo adattarsi ai dati esistenti, ma anche generalizzare bene su dati futuri.

::: {#exm-}

Consideriamo un secondo esempio per illustrare il concetto di ELPD utilizzando la distribuzione binomiale. Immaginiamo di condurre un esperimento in cui lanciamo una moneta 10 volte e contiamo il numero di volte in cui otteniamo testa. Supponiamo che la vera probabilità di ottenere testa sia 0.6.

1) *Distribuzione reale dei dati:* Segue una distribuzione binomiale con 10 lanci e probabilità di successo pari a 0.6: $y \sim \text{Binomiale}(10, 0.6).$
2) *Distribuzione stimata dal modello:* Il nostro modello ipotizza che la probabilità di ottenere testa sia 0.5, cioè considera la moneta come equa: $p(y \mid \theta) = \text{Binomiale}(10, 0.5).$

Ora procediamo al calcolo dell'ELPD.

In [4]:
# Parametri
n = 10  # numero di lanci
p = 0.6  # vera probabilità di testa
q = 0.5  # probabilità stimata dal modello

# Calcolo ELPD
elpd = 0
for y in range(n + 1):
    # Probabilità di y secondo la vera distribuzione
    p_y = binom.pmf(y, n, p)

    # Log della densità predittiva del modello
    log_q_y = binom.logpmf(y, n, q)

    # Somma pesata
    elpd += p_y * log_q_y

print(f"ELPD del modello che stima p=0.5: {elpd:.4f}")

# Per confronto, calcoliamo l'ELPD per il modello "vero"
elpd_true = 0
for y in range(n + 1):
    p_y = binom.pmf(y, n, p)
    log_p_y = binom.logpmf(y, n, p)
    elpd_true += p_y * log_p_y

print(f"ELPD del modello vero (con p=0.6): {elpd_true:.4f}")

ELPD del modello che stima p=0.5: -2.0549
ELPD del modello vero (con p=0.6): -1.8536


La conclusione è che l'ELPD del modello vero è maggiore (meno negativo) rispetto a quello del nostro modello stimato, il che riflette una capacità predittiva superiore del modello vero.

Questo esempio dimostra come l'ELPD quantifica la capacità predittiva di un modello:

1. Si considerano tutti i possibili risultati dell'esperimento (da 0 a 10 teste).
2. Per ciascun risultato, si calcola:

   - La probabilità di osservare quel risultato secondo la distribuzione reale.
   - Il logaritmo della densità predittiva del modello stimato $q$ per lo stesso risultato.
3. Si moltiplicano questi due valori e si sommano i risultati per tutti i possibili esiti.

In sintesi, l'ELPD permette di confrontare l'efficacia predittiva di diversi modelli: un valore più alto (meno negativo) indica una migliore capacità del modello di prevedere i dati osservati. Nell'esempio presentato, il modello vero (con $p = 0.6$) ha un ELPD superiore rispetto al modello stimato (con $p = 0.5$), confermando che il primo ha una capacità predittiva migliore.

:::

## Metodi di Approssimazione per la Stima dell'ELPD

Come la divergenza KL, anche il calcolo dell'ELPD richiede, in teoria, la conoscenza della vera distribuzione $P$, che è ignota. Tuttavia, a differenza della divergenza KL, disponiamo di metodi pratici per approssimare l'ELPD. Uno dei metodi più robusti e comunemente utilizzati per questa stima è la Leave-One-Out Cross-Validation (LOO-CV).

### Leave-One-Out Cross-Validation (LOO-CV)

Il LOO-CV è una tecnica che consente di stimare l'ELPD senza la necessità di conoscere la vera distribuzione dei dati. Questo metodo consiste nel rimuovere una singola osservazione dal dataset, adattare il modello sui dati rimanenti, e poi valutare la densità predittiva per l'osservazione esclusa. Questa procedura viene ripetuta per ogni osservazione nel dataset, e i risultati vengono sommati per ottenere una stima complessiva dell'ELPD.

Procedura:

1. rimuovere una singola osservazione $y_i$ dal dataset;
2. adattare il modello ai dati rimanenti $y_{-i}$;
3. calcolare la densità predittiva $p(y_i \mid y_{-i})$ per l'osservazione esclusa;
4. ripetere il processo per tutte le osservazioni e sommare i logaritmi delle densità predittive per ottenere l'ELPD.

Formula:

$$ \text{Stima dell'ELPD} = \sum_{i=1}^{N} \log p(y_i \mid y_{-i}), $$

dove $y_i$ è il dato escluso e $y_{-i}$ rappresenta tutti gli altri dati.

Vantaggi e limitazioni:

- LOO-CV fornisce una stima robusta dell'ELPD, particolarmente utile per modelli complessi e dataset di dimensioni ridotte;
- questo metodo è computazionalmente intensivo, specialmente per dataset molto grandi o modelli complessi.

### Criteri di Informazione come Approssimazioni alla Divergenza KL

Oltre al LOO-CV, sono stati proposti altri metodi per approssimare l'ELPD, che derivano direttamente dal concetto di divergenza KL. Tra questi, i più noti sono i criteri di informazione, come l'errore quadratico medio (MSE), l'Akaike Information Criterion (AIC), il Bayesian Information Criterion (BIC) e il Widely Applicable Information Criterion (WAIC).

Questi criteri forniscono approcci alternativi per valutare i modelli, bilanciando la bontà di adattamento del modello ai dati osservati con la sua complessità. 

#### Errore Quadratico Medio (MSE)

L'Errore Quadratico Medio (Mean Squared Error o MSE) misura la discrepanza media tra le previsioni del modello e i valori reali.

Formula:

$$ MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2, $$

dove:

- $n$ è il numero totale di osservazioni,
- $y_i$ sono i valori reali,
- $\hat{y}_i$ sono i valori previsti dal modello.

Un MSE inferiore indica un migliore adattamento del modello ai dati.

#### Criterio di Informazione di Akaike (AIC)

Il Criterio di Informazione di Akaike (AIC) va oltre l'MSE, considerando sia l'adattamento del modello che la sua complessità.

Formula:

$$ AIC = -2 \sum \log p(y_i \mid \hat{\theta}_{\text{mle}}) + 2k, $$

dove:

- $\hat{\theta}_{\text{mle}}$ sono i parametri stimati del modello,
- $k$ è il numero di parametri del modello.

L'AIC bilancia la bontà di adattamento (primo termine) con la complessità del modello (secondo termine). Un valore più basso di AIC indica una minor perdita di informazione, suggerendo un modello preferibile.

Vantaggi e limitazioni:

- facile e veloce da calcolare;
- può essere meno accurato per campioni piccoli o modelli complessi;
- fornisce un'approssimazione asintoticamente corretta dell'ELPD per modelli regolari e campioni grandi.

#### Criterio di Informazione Bayesiano (BIC)

Il Criterio di Informazione Bayesiano (BIC) è definito come:

$$
BIC = \ln(n)k - 2\ln(L),
$$

dove:

- $n$ è il numero di osservazioni,
- $k$ è il numero di parametri del modello,
- $L$ è il valore massimo della funzione di verosimiglianza del modello.

Il termine $\ln(L)$ rappresenta il logaritmo naturale della verosimiglianza massima, che indica quanto bene il modello si adatta ai dati osservati.
  
Il BIC impone una penalità maggiore per l'incremento dei parametri, rendendolo particolarmente adeguato per dataset di grandi dimensioni.

#### Widely Applicable Information Criterion (WAIC)

Il WAIC è una versione avanzata dell'AIC, particolarmente utile nel contesto bayesiano. Considera l'intera distribuzione a posteriori dei parametri anziché solo la stima puntuale.

Formula:

$$ 
WAIC = -2\left[ \sum_{i=1}^{n} \log \left( \frac{1}{S} \sum_{s=1}^{S} p(y_i \mid \theta^{(s)}) \right) - \sum_{i=1}^{n} \text{Var}_{\theta^{(s)}} \left( \log p(y_i \mid \theta^{(s)}) \right) \right], 
$$

dove:

- $S$ è il numero di campioni dalla distribuzione a posteriori,
- $\text{Var}_{\theta^{(s)}}$ è la varianza della log-verosimiglianza.

Caratteristiche del WAIC:

1. calcola il logaritmo della densità predittiva per ogni punto dati;
2. penalizza la complessità del modello basandosi sulla variabilità delle sue predizioni;
3. la somma delle varianze a posteriori del logaritmo della densità predittiva converge al numero effettivo di parametri del modello.

## Considerazioni Conclusive

La valutazione dei modelli statistici è essenziale per garantire sia la loro affidabilità che la loro capacità di generalizzazione. Quando si sviluppano modelli, è fondamentale disporre di strumenti che permettano di confrontare diverse alternative e selezionare quella più adatta ai dati, garantendo al contempo previsioni accurate. In questo contesto, la divergenza di Kullback-Leibler (KL), l'Expected Log-Predictive Density (ELPD) e i criteri di informazione come AIC, BIC e WAIC offrono un quadro completo per la valutazione delle performance dei modelli.

La divergenza KL rappresenta una misura ideale per quantificare la distanza tra la distribuzione di probabilità reale dei dati e quella stimata dal modello. Tuttavia, la sua applicazione pratica è limitata poiché richiede la conoscenza della distribuzione vera dei dati, che è generalmente sconosciuta. Questa limitazione rende impossibile calcolarla direttamente nella maggior parte dei casi pratici.

Per superare questo ostacolo, si utilizza l'Expected Log-Predictive Density (ELPD). L'ELPD fornisce una stima della capacità predittiva del modello su nuovi dati, ovvero la sua abilità di generalizzare oltre i dati di training. Un'approssimazione particolarmente utile dell'ELPD si ottiene tramite la Leave-One-Out Cross-Validation (LOO-CV), che consiste nel rimuovere iterativamente un'osservazione dal dataset, addestrare il modello sui dati rimanenti e prevedere il valore dell'osservazione omessa. Questo processo permette di stimare accuratamente la capacità predittiva del modello su ogni singola osservazione, fornendo così un'approssimazione precisa dell'ELPD.

L'ELPD e la divergenza KL sono strumenti complementari per la valutazione dei modelli statistici:

1. l'ELPD misura la capacità predittiva su nuovi dati, con valori più alti che indicano previsioni migliori e una maggiore capacità di generalizzazione del modello.
2. la divergenza KL quantifica la differenza tra la distribuzione vera dei dati e quella stimata dal modello, con valori più bassi che indicano una migliore approssimazione della distribuzione reale da parte del modello.

Esiste una relazione diretta tra ELPD e divergenza KL: massimizzare l'ELPD equivale a minimizzare la divergenza KL. Entrambi gli approcci mirano a sviluppare modelli che catturino al meglio la realtà rappresentata dai dati.

In pratica:

- la divergenza KL valuta l'adattamento del modello ai dati osservati, fornendo una misura di quanto il modello riesca a replicare la distribuzione reale;
- l'ELPD e i suoi metodi di approssimazione (come LOO-CV, AIC, WAIC) misurano la capacità del modello di generalizzare su dati futuri, offrendo una valutazione più completa della qualità del modello.

Anche i criteri di informazione come AIC, BIC e WAIC sono strumenti utili per approssimare la divergenza KL. Essi permettono di valutare i modelli in termini di adattamento ai dati e complessità, facilitando la selezione di un modello che bilancia precisione predittiva e parsimonia.

Ciascun metodo fornisce prospettive diverse e complementari per sostituire la divergenza KL. Il LOO-CV è particolarmente utile per modelli complessi o quando si dispone di set di dati limitati, poiché offre una valutazione robusta, sebbene computazionalmente intensiva. D'altro canto, l'AIC e il WAIC sono più veloci e meno onerosi dal punto di vista computazionale, rendendoli adatti per valutazioni preliminari o quando si lavora con grandi dataset.

In conclusione, la selezione del modello ottimale richiede un equilibrio tra adattamento ai dati e semplicità. L'uso combinato di tecniche di validazione incrociata e criteri di informazione permette di costruire modelli che:

1. si adattano bene ai dati osservati,
2. forniscono previsioni affidabili su nuovi dati,
3. catturano le tendenze rilevanti senza essere eccessivamente influenzati dal rumore.

L'obiettivo finale non è creare il modello più complesso o quello che si adatta perfettamente ai dati di addestramento, ma piuttosto trovare un equilibrio ottimale tra semplicità e accuratezza, garantendo che il modello sia robusto e generalizzabile a dati futuri.

## Informazioni sull'Ambiente di Sviluppo {.unnumbered}

In [25]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Fri Jul 26 2024

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

arviz     : 0.18.0
matplotlib: 3.9.1
numpy     : 1.26.4
scipy     : 1.14.0

Watermark: 2.4.3

